In [0]:
#import modules we need to use for this session
import pandas as pd
import numpy as np
%matplotlib inline
import matplotlib.pyplot as plt
from scipy import stats
import statsmodels.formula.api as smf
import statsmodels.api as sm
from sklearn.linear_model import LinearRegression
# PCA module
from sklearn.decomposition import PCA
from mpl_toolkits.mplot3d import Axes3D
from sklearn import datasets
from sklearn import linear_model
from sklearn import preprocessing
from sklearn.metrics import r2_score

In [125]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# Task 1. House price prediction using 311 data

### Can 311 complaining profile of urban neighborhoods enhance real-estate price prediction?

[Zhu, E., & Sobolevsky, S. (2018). House Price Modeling with Digital Census. arXiv preprint arXiv:1809.03834.]

Before we tried to use some parameters of the house (size) to predict its price. However notice that location matters as well. Below we upload a sample of more than 14000 individual house sales all around NYC in 2012. Each record together with the parameters of the house also contains important characteristics of the location (zip code) - average income of its residents (accodring to US census), as well as the relative structure of 311 complaints happening in the area.

In [0]:
REsample=pd.read_csv('/content/drive/My Drive/Untitled folder/NYChome_sales2012.csv',index_col=0)

In [127]:
len(REsample)

14611

In [128]:
REsample.head()

,borough,neighborhood,block,lot,easement,building_class_present,address,zip_code,commmercial_units,land_sq_feet,gross_sq_feet,year_built,sale_price,sale_date
204018,4,LAURELTON,12941,94,NaN,A2,131-43 FRANCIS LEWIS BLVD,11413,0,3560,1473,1940,0,2012-01-01
217555,2,BATHGATE,3046,42,NaN,A1,2069 BATHGATE AVE,10457,0,1964,1424,1899,345376,2012-04-05
217598,2,BAYCHESTER,4707,70,NaN,A5,1134 EAST 213 STREET,10469,0,2320,1813,1950,335700,2012-05-17
217599,2,BAYCHESTER,4708,48,NaN,A5,1110 EAST 214TH STREET,10469,0,1710,1813,1945,338000,2012-02-03
217600,2,BAYCHESTER,4710,1,NaN,A5,1127 EAST 215TH STREET,10469,0,2759,1998,1945,0,2012-12-20


In [129]:
REsample.columns

Index([u'borough', u'neighborhood', u'block', u'lot', u'easement',
       u'building_class_present', u'address', u'zip_code',
       u'commmercial_units', u'land_sq_feet', u'gross_sq_feet', u'year_built',
       u'sale_price', u'sale_date'],
      dtype='object')

In [130]:
#how many houses do we have per each borough?
np.bincount(REsample.borough)

array([   0,  216,  894, 2769, 7288, 3444])

In [131]:
#split into training (1), validation (2) and test (3) samples
np.random.seed(2018)
splitind=np.random.uniform(size=len(REsample)) #draw samples within low and high limit, each sample is likely to be drawn 
splitind

array([0.88234931, 0.10432774, 0.90700933, ..., 0.86845249, 0.57514979,
       0.79229244])

In [0]:
REsample1=REsample.loc[splitind<=0.4] #training
REsample2=REsample.loc[(splitind>0.4)&(splitind<0.7)] #validation
REsample3=REsample.loc[(splitind>=0.7)] #test

In [133]:
len(REsample1) #loosing around 50 records within sparse zip codes

5832

In [134]:
len(REsample2)

4394

In [135]:
len(REsample3)

4385

In [0]:
# now load and curate 311 data which we're going to use in order to characterize the location

In [137]:
data311 = pd.read_csv('/content/drive/My Drive/Untitled folder/aggr311.csv' , index_col=0 )
data311.head()

,Zip,Complain,Count
1,NaN,Adopt-A-Basket,5
2,10001,Adopt-A-Basket,1
3,10003,Adopt-A-Basket,1
4,10009,Adopt-A-Basket,1
5,10010,Adopt-A-Basket,1


In [138]:
data311.Zip=pd.to_numeric(data311.Zip,errors='coerce') #Take separate series and convert to numeric, coercing when told to, errors = 'coerce' means then invalid parsing will be set as NaN
data311.head()

,Zip,Complain,Count
1,NaN,Adopt-A-Basket,5
2,10001.0,Adopt-A-Basket,1
3,10003.0,Adopt-A-Basket,1
4,10009.0,Adopt-A-Basket,1
5,10010.0,Adopt-A-Basket,1


In [0]:
data311=data311.loc[(data311.Zip>=10000)&(data311.Zip<11500)] #take only NYC zip codes

In [0]:
data311=pd.pivot_table(data311,index='Zip',columns='Complain',values='Count',fill_value=0)

In [141]:
data311.head()

Complain,APPLIANCE,Adopt-A-Basket,Air Quality,Animal Abuse,Animal Facility - No Permit,Animal in a Park,Asbestos,BEST/Site Safety,Beach/Pool/Sauna Complaint,Bike Rack Condition,Bike/Roller/Skate Chronic,Blocked Driveway,Boilers,Bottled Water,Bridge Condition,Broken Muni Meter,Broken Parking Meter,Building Condition,Building/Use,Bus Stop Shelter Placement,CONSTRUCTION,Calorie Labeling,City Vehicle Placard Complaint,Collection Truck Noise,Construction,Consumer Complaint,Cranes and Derricks,Curb Condition,DOF Parking - Tax Exemption,DPR Internal,Damaged Tree,Dead Tree,Derelict Bicycle,Derelict Vehicle,Derelict Vehicles,Dirty Conditions,Disorderly Youth,Drinking,Drinking Water,EAP Inspection - F59,...,School Maintenance,Senior Center Complaint,Sewer,Sidewalk Condition,Smoking,Snow,Special Enforcement,Special Natural Area District (SNAD),Special Projects Inspection Team (SPIT),Sprinkler - Mechanical,Squeegee,Stalled Sites,Standing Water,Standpipe - Mechanical,Street Condition,Street Light Condition,Street Sign - Damaged,Street Sign - Dangling,Street Sign - Missing,Summer Camp,Sweeping/Missed-Inadequate,Tattooing,Taxi Complaint,Traffic,Traffic Signal Condition,Trans Fat,Transportation Provider Complaint,Unleashed Dog,Unsanitary Animal Facility,Unsanitary Animal Pvt Property,Unsanitary Pigeon Condition,Urinating in Public,Vacant Lot,Vending,Violation of Park Rules,Water Conservation,Water Quality,Water System,Window Guard,X-Ray Machine/Equipment
Zip,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000.0,0,0,1,0,0,1,0,0,0,0,8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,1,2,0,0,0,0,1,3,0,0,0,...,0,2,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,1,0,0,0,16,1,0,0,0,0,0,0,0,0,0,8,8,0,0,0,0,0
10001.0,0,1,90,0,0,1,20,36,1,1,2,51,2,1,0,498,1,0,33,3,0,0,3,1,3,394,18,0,5,2,7,7,4,7,3,108,0,7,1,98,...,2,7,71,49,29,9,38,0,38,0,0,0,5,0,429,4,14,1,16,0,1,0,707,39,1,0,0,0,0,0,3,0,2,51,5,5,2,366,0,0
10002.0,15,0,80,0,0,21,18,20,2,6,8,73,8,0,24,921,2,0,120,4,2,0,4,2,0,167,4,4,9,5,30,14,20,17,8,144,4,14,1,0,...,19,2,102,50,27,7,26,0,65,0,0,1,8,0,408,296,32,8,14,1,4,4,368,28,319,0,0,1,0,7,5,1,4,27,14,10,7,324,0,0
10003.0,15,1,143,0,2,44,24,13,2,2,14,69,16,0,0,1461,1,0,67,4,4,2,4,3,3,181,4,3,17,5,45,11,33,17,16,162,5,21,0,25,...,17,5,115,65,21,7,45,0,49,1,0,0,3,1,572,32,10,16,16,0,5,0,600,20,23,0,0,2,1,7,7,11,2,54,25,12,5,318,0,0
10004.0,0,0,15,0,0,4,3,1,0,1,1,8,3,0,0,123,0,0,3,0,0,0,2,0,0,55,0,2,2,0,1,1,0,1,1,12,0,0,0,19,...,5,0,32,23,8,1,8,0,3,0,0,0,1,0,133,93,39,17,21,0,0,0,142,10,96,0,1,0,0,0,1,1,0,16,21,0,0,37,0,0


In [142]:
list_311=list(data311.columns)
list_311[:10]

['APPLIANCE',
 'Adopt-A-Basket',
 'Air Quality',
 'Animal Abuse',
 'Animal Facility - No Permit',
 'Animal in a Park',
 'Asbestos',
 'BEST/Site Safety',
 'Beach/Pool/Sauna Complaint',
 'Bike Rack Condition']

In [143]:
Total311=data311.sum(axis=1) #total 311 activity per zip code
Total311.head()

Zip
10000.0      114
10001.0     6829
10002.0    11276
10003.0    11269
10004.0     1540
dtype: int64

In [144]:
data311.head()

Complain,APPLIANCE,Adopt-A-Basket,Air Quality,Animal Abuse,Animal Facility - No Permit,Animal in a Park,Asbestos,BEST/Site Safety,Beach/Pool/Sauna Complaint,Bike Rack Condition,Bike/Roller/Skate Chronic,Blocked Driveway,Boilers,Bottled Water,Bridge Condition,Broken Muni Meter,Broken Parking Meter,Building Condition,Building/Use,Bus Stop Shelter Placement,CONSTRUCTION,Calorie Labeling,City Vehicle Placard Complaint,Collection Truck Noise,Construction,Consumer Complaint,Cranes and Derricks,Curb Condition,DOF Parking - Tax Exemption,DPR Internal,Damaged Tree,Dead Tree,Derelict Bicycle,Derelict Vehicle,Derelict Vehicles,Dirty Conditions,Disorderly Youth,Drinking,Drinking Water,EAP Inspection - F59,...,School Maintenance,Senior Center Complaint,Sewer,Sidewalk Condition,Smoking,Snow,Special Enforcement,Special Natural Area District (SNAD),Special Projects Inspection Team (SPIT),Sprinkler - Mechanical,Squeegee,Stalled Sites,Standing Water,Standpipe - Mechanical,Street Condition,Street Light Condition,Street Sign - Damaged,Street Sign - Dangling,Street Sign - Missing,Summer Camp,Sweeping/Missed-Inadequate,Tattooing,Taxi Complaint,Traffic,Traffic Signal Condition,Trans Fat,Transportation Provider Complaint,Unleashed Dog,Unsanitary Animal Facility,Unsanitary Animal Pvt Property,Unsanitary Pigeon Condition,Urinating in Public,Vacant Lot,Vending,Violation of Park Rules,Water Conservation,Water Quality,Water System,Window Guard,X-Ray Machine/Equipment
Zip,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000.0,0,0,1,0,0,1,0,0,0,0,8,1,0,0,0,0,0,0,0,0,0,0,0,0,0,4,0,0,0,1,2,0,0,0,0,1,3,0,0,0,...,0,2,0,0,0,0,0,0,0,0,0,0,0,0,6,0,0,0,1,0,0,0,16,1,0,0,0,0,0,0,0,0,0,8,8,0,0,0,0,0
10001.0,0,1,90,0,0,1,20,36,1,1,2,51,2,1,0,498,1,0,33,3,0,0,3,1,3,394,18,0,5,2,7,7,4,7,3,108,0,7,1,98,...,2,7,71,49,29,9,38,0,38,0,0,0,5,0,429,4,14,1,16,0,1,0,707,39,1,0,0,0,0,0,3,0,2,51,5,5,2,366,0,0
10002.0,15,0,80,0,0,21,18,20,2,6,8,73,8,0,24,921,2,0,120,4,2,0,4,2,0,167,4,4,9,5,30,14,20,17,8,144,4,14,1,0,...,19,2,102,50,27,7,26,0,65,0,0,1,8,0,408,296,32,8,14,1,4,4,368,28,319,0,0,1,0,7,5,1,4,27,14,10,7,324,0,0
10003.0,15,1,143,0,2,44,24,13,2,2,14,69,16,0,0,1461,1,0,67,4,4,2,4,3,3,181,4,3,17,5,45,11,33,17,16,162,5,21,0,25,...,17,5,115,65,21,7,45,0,49,1,0,0,3,1,572,32,10,16,16,0,5,0,600,20,23,0,0,2,1,7,7,11,2,54,25,12,5,318,0,0
10004.0,0,0,15,0,0,4,3,1,0,1,1,8,3,0,0,123,0,0,3,0,0,0,2,0,0,55,0,2,2,0,1,1,0,1,1,12,0,0,0,19,...,5,0,32,23,8,1,8,0,3,0,0,0,1,0,133,93,39,17,21,0,0,0,142,10,96,0,1,0,0,0,1,1,0,16,21,0,0,37,0,0


In [145]:
data311=data311.div(data311.sum(axis=1), axis=0) #normalize activity of various categories within zip code by total
data311=data311.loc[Total311>100] #keep only those zip codes having sufficient activity
data311.head()

Complain,APPLIANCE,Adopt-A-Basket,Air Quality,Animal Abuse,Animal Facility - No Permit,Animal in a Park,Asbestos,BEST/Site Safety,Beach/Pool/Sauna Complaint,Bike Rack Condition,Bike/Roller/Skate Chronic,Blocked Driveway,Boilers,Bottled Water,Bridge Condition,Broken Muni Meter,Broken Parking Meter,Building Condition,Building/Use,Bus Stop Shelter Placement,CONSTRUCTION,Calorie Labeling,City Vehicle Placard Complaint,Collection Truck Noise,Construction,Consumer Complaint,Cranes and Derricks,Curb Condition,DOF Parking - Tax Exemption,DPR Internal,Damaged Tree,Dead Tree,Derelict Bicycle,Derelict Vehicle,Derelict Vehicles,Dirty Conditions,Disorderly Youth,Drinking,Drinking Water,EAP Inspection - F59,...,School Maintenance,Senior Center Complaint,Sewer,Sidewalk Condition,Smoking,Snow,Special Enforcement,Special Natural Area District (SNAD),Special Projects Inspection Team (SPIT),Sprinkler - Mechanical,Squeegee,Stalled Sites,Standing Water,Standpipe - Mechanical,Street Condition,Street Light Condition,Street Sign - Damaged,Street Sign - Dangling,Street Sign - Missing,Summer Camp,Sweeping/Missed-Inadequate,Tattooing,Taxi Complaint,Traffic,Traffic Signal Condition,Trans Fat,Transportation Provider Complaint,Unleashed Dog,Unsanitary Animal Facility,Unsanitary Animal Pvt Property,Unsanitary Pigeon Condition,Urinating in Public,Vacant Lot,Vending,Violation of Park Rules,Water Conservation,Water Quality,Water System,Window Guard,X-Ray Machine/Equipment
Zip,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000.0,0.000000,0.000000,0.008772,0.0,0.000000,0.008772,0.000000,0.000000,0.000000,0.000000,0.070175,0.008772,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035088,0.000000,0.000000,0.000000,0.008772,0.017544,0.000000,0.000000,0.000000,0.000000,0.008772,0.026316,0.000000,0.000000,0.000000,...,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.008772,0.000000,0.000000,0.000000,0.140351,0.008772,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.070175,0.070175,0.000000,0.000000,0.000000,0.0,0.0
10001.0,0.000000,0.000146,0.013179,0.0,0.000000,0.000146,0.002929,0.005272,0.000146,0.000146,0.000293,0.007468,0.000293,0.000146,0.000000,0.072924,0.000146,0.0,0.004832,0.000439,0.000000,0.000000,0.000439,0.000146,0.000439,0.057695,0.002636,0.000000,0.000732,0.000293,0.001025,0.001025,0.000586,0.001025,0.000439,0.015815,0.000000,0.001025,0.000146,0.014351,...,0.000293,0.001025,0.010397,0.007175,0.004247,0.001318,0.005565,0.0,0.005565,0.000000,0.0,0.000000,0.000732,0.000000,0.062820,0.000586,0.002050,0.000146,0.002343,0.000000,0.000146,0.000000,0.103529,0.005711,0.000146,0.0,0.000000,0.000000,0.000000,0.000000,0.000439,0.000000,0.000293,0.007468,0.000732,0.000732,0.000293,0.053595,0.0,0.0
10002.0,0.001330,0.000000,0.007095,0.0,0.000000,0.001862,0.001596,0.001774,0.000177,0.000532,0.000709,0.006474,0.000709,0.000000,0.002128,0.081678,0.000177,0.0,0.010642,0.000355,0.000177,0.000000,0.000355,0.000177,0.000000,0.014810,0.000355,0.000355,0.000798,0.000443,0.002661,0.001242,0.001774,0.001508,0.000709,0.012770,0.000355,0.001242,0.000089,0.000000,...,0.001685,0.000177,0.009046,0.004434,0.002394,0.000621,0.002306,0.0,0.005764,0.000000,0.0,0.000089,0.000709,0.000000,0.036183,0.026250,0.002838,0.000709,0.001242,0.000089,0.000355,0.000355,0.032636,0.002483,0.028290,0.0,0.000000,0.000089,0.000000,0.000621,0.000443,0.000089,0.000355,0.002394,0.001242,0.000887,0.000621,0.028734,0.0,0.0
10003.0,0.001331,0.000089,0.012690,0.0,0.000177,0.003905,0.002130,0.001154,0.000177,0.000177,0.001242,0.006123,0.001420,0.000000,0.000000,0.129648,0.000089,0.0,0.005946,0.000355,0.000355,0.000177,0.000355,0.000266,0.000266,0.016062,0.000355,0.000266,0.001509,0.000444,0.003993,0.000976,0.002928,0.001509,0.001420,0.014376,0.000444,0.001864,0.000000,0.002218,...

In [146]:
len(data311) #only 179 zip codes with reliable data left

179

In [147]:
data311.head()

Complain,APPLIANCE,Adopt-A-Basket,Air Quality,Animal Abuse,Animal Facility - No Permit,Animal in a Park,Asbestos,BEST/Site Safety,Beach/Pool/Sauna Complaint,Bike Rack Condition,Bike/Roller/Skate Chronic,Blocked Driveway,Boilers,Bottled Water,Bridge Condition,Broken Muni Meter,Broken Parking Meter,Building Condition,Building/Use,Bus Stop Shelter Placement,CONSTRUCTION,Calorie Labeling,City Vehicle Placard Complaint,Collection Truck Noise,Construction,Consumer Complaint,Cranes and Derricks,Curb Condition,DOF Parking - Tax Exemption,DPR Internal,Damaged Tree,Dead Tree,Derelict Bicycle,Derelict Vehicle,Derelict Vehicles,Dirty Conditions,Disorderly Youth,Drinking,Drinking Water,EAP Inspection - F59,...,School Maintenance,Senior Center Complaint,Sewer,Sidewalk Condition,Smoking,Snow,Special Enforcement,Special Natural Area District (SNAD),Special Projects Inspection Team (SPIT),Sprinkler - Mechanical,Squeegee,Stalled Sites,Standing Water,Standpipe - Mechanical,Street Condition,Street Light Condition,Street Sign - Damaged,Street Sign - Dangling,Street Sign - Missing,Summer Camp,Sweeping/Missed-Inadequate,Tattooing,Taxi Complaint,Traffic,Traffic Signal Condition,Trans Fat,Transportation Provider Complaint,Unleashed Dog,Unsanitary Animal Facility,Unsanitary Animal Pvt Property,Unsanitary Pigeon Condition,Urinating in Public,Vacant Lot,Vending,Violation of Park Rules,Water Conservation,Water Quality,Water System,Window Guard,X-Ray Machine/Equipment
Zip,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000.0,0.000000,0.000000,0.008772,0.0,0.000000,0.008772,0.000000,0.000000,0.000000,0.000000,0.070175,0.008772,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035088,0.000000,0.000000,0.000000,0.008772,0.017544,0.000000,0.000000,0.000000,0.000000,0.008772,0.026316,0.000000,0.000000,0.000000,...,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.008772,0.000000,0.000000,0.000000,0.140351,0.008772,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.070175,0.070175,0.000000,0.000000,0.000000,0.0,0.0
10001.0,0.000000,0.000146,0.013179,0.0,0.000000,0.000146,0.002929,0.005272,0.000146,0.000146,0.000293,0.007468,0.000293,0.000146,0.000000,0.072924,0.000146,0.0,0.004832,0.000439,0.000000,0.000000,0.000439,0.000146,0.000439,0.057695,0.002636,0.000000,0.000732,0.000293,0.001025,0.001025,0.000586,0.001025,0.000439,0.015815,0.000000,0.001025,0.000146,0.014351,...,0.000293,0.001025,0.010397,0.007175,0.004247,0.001318,0.005565,0.0,0.005565,0.000000,0.0,0.000000,0.000732,0.000000,0.062820,0.000586,0.002050,0.000146,0.002343,0.000000,0.000146,0.000000,0.103529,0.005711,0.000146,0.0,0.000000,0.000000,0.000000,0.000000,0.000439,0.000000,0.000293,0.007468,0.000732,0.000732,0.000293,0.053595,0.0,0.0
10002.0,0.001330,0.000000,0.007095,0.0,0.000000,0.001862,0.001596,0.001774,0.000177,0.000532,0.000709,0.006474,0.000709,0.000000,0.002128,0.081678,0.000177,0.0,0.010642,0.000355,0.000177,0.000000,0.000355,0.000177,0.000000,0.014810,0.000355,0.000355,0.000798,0.000443,0.002661,0.001242,0.001774,0.001508,0.000709,0.012770,0.000355,0.001242,0.000089,0.000000,...,0.001685,0.000177,0.009046,0.004434,0.002394,0.000621,0.002306,0.0,0.005764,0.000000,0.0,0.000089,0.000709,0.000000,0.036183,0.026250,0.002838,0.000709,0.001242,0.000089,0.000355,0.000355,0.032636,0.002483,0.028290,0.0,0.000000,0.000089,0.000000,0.000621,0.000443,0.000089,0.000355,0.002394,0.001242,0.000887,0.000621,0.028734,0.0,0.0
10003.0,0.001331,0.000089,0.012690,0.0,0.000177,0.003905,0.002130,0.001154,0.000177,0.000177,0.001242,0.006123,0.001420,0.000000,0.000000,0.129648,0.000089,0.0,0.005946,0.000355,0.000355,0.000177,0.000355,0.000266,0.000266,0.016062,0.000355,0.000266,0.001509,0.000444,0.003993,0.000976,0.002928,0.001509,0.001420,0.014376,0.000444,0.001864,0.000000,0.002218,...

In [0]:
data311.columns  = data311.columns.str.lower()
data311.columns  =data311.columns.str.split(' ').str.get(0)

In [149]:
data311.head()

Complain,appliance,adopt-a-basket,air,animal,animal,animal,asbestos,best/site,beach/pool/sauna,bike,bike/roller/skate,blocked,boilers,bottled,bridge,broken,broken,building,building/use,bus,construction,calorie,city,collection,construction,consumer,cranes,curb,dof,dpr,damaged,dead,derelict,derelict,derelict,dirty,disorderly,drinking,drinking,eap,...,school,senior,sewer,sidewalk,smoking,snow,special,special,special,sprinkler,squeegee,stalled,standing,standpipe,street,street,street,street,street,summer,sweeping/missed-inadequate,tattooing,taxi,traffic,traffic,trans,transportation,unleashed,unsanitary,unsanitary,unsanitary,urinating,vacant,vending,violation,water,water,water,window,x-ray
Zip,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
10000.0,0.000000,0.000000,0.008772,0.0,0.000000,0.008772,0.000000,0.000000,0.000000,0.000000,0.070175,0.008772,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.035088,0.000000,0.000000,0.000000,0.008772,0.017544,0.000000,0.000000,0.000000,0.000000,0.008772,0.026316,0.000000,0.000000,0.000000,...,0.000000,0.017544,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.052632,0.000000,0.000000,0.000000,0.008772,0.000000,0.000000,0.000000,0.140351,0.008772,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.070175,0.070175,0.000000,0.000000,0.000000,0.0,0.0
10001.0,0.000000,0.000146,0.013179,0.0,0.000000,0.000146,0.002929,0.005272,0.000146,0.000146,0.000293,0.007468,0.000293,0.000146,0.000000,0.072924,0.000146,0.0,0.004832,0.000439,0.000000,0.000000,0.000439,0.000146,0.000439,0.057695,0.002636,0.000000,0.000732,0.000293,0.001025,0.001025,0.000586,0.001025,0.000439,0.015815,0.000000,0.001025,0.000146,0.014351,...,0.000293,0.001025,0.010397,0.007175,0.004247,0.001318,0.005565,0.0,0.005565,0.000000,0.0,0.000000,0.000732,0.000000,0.062820,0.000586,0.002050,0.000146,0.002343,0.000000,0.000146,0.000000,0.103529,0.005711,0.000146,0.0,0.000000,0.000000,0.000000,0.000000,0.000439,0.000000,0.000293,0.007468,0.000732,0.000732,0.000293,0.053595,0.0,0.0
10002.0,0.001330,0.000000,0.007095,0.0,0.000000,0.001862,0.001596,0.001774,0.000177,0.000532,0.000709,0.006474,0.000709,0.000000,0.002128,0.081678,0.000177,0.0,0.010642,0.000355,0.000177,0.000000,0.000355,0.000177,0.000000,0.014810,0.000355,0.000355,0.000798,0.000443,0.002661,0.001242,0.001774,0.001508,0.000709,0.012770,0.000355,0.001242,0.000089,0.000000,...,0.001685,0.000177,0.009046,0.004434,0.002394,0.000621,0.002306,0.0,0.005764,0.000000,0.0,0.000089,0.000709,0.000000,0.036183,0.026250,0.002838,0.000709,0.001242,0.000089,0.000355,0.000355,0.032636,0.002483,0.028290,0.0,0.000000,0.000089,0.000000,0.000621,0.000443,0.000089,0.000355,0.002394,0.001242,0.000887,0.000621,0.028734,0.0,0.0
10003.0,0.001331,0.000089,0.012690,0.0,0.000177,0.003905,0.002130,0.001154,0.000177,0.000177,0.001242,0.006123,0.001420,0.000000,0.000000,0.129648,0.000089,0.0,0.005946,0.000355,0.000355,0.000177,0.000355,0.000266,0.000266,0.016062,0.000355,0.000266,0.001509,0.000444,0.003993,0.000976,0.002928,0.001509,0.001420,0.014376,0.000444,0.001864,0.000000,0.002218,...,0.001509,0.000444,0.010205,0.005768,0.001864,0.000621,0.003993,0.0,0.004348,0.000089,0.0,0.000000,0.000266,0.000089,0.050759,0.002840,0.000887,0.001420,0.001420,0.000000,0.000444,0.000000,0.053243,0.001775,0.002041,0.0,0.000000,0.000177,0.000089,0.000621,0.000621,0.000976,0.000177,0.004792,0.002218,0.001065,0.000444,0.028219,0.0,0.0
10004.0,0.000000,0.000000,0.009740,0.0,0.000000,0.002597,0.001948,0.000649,0.000000,0.000649,0.000649,0.005195,0.001948,0.000000,0.000000,0.079870,0.000000,0.0,0.001948,0.000000,0.000000,0.000000,0.001299,0.000000,0.000000,0.035714,0.000000,0.001299,0.001299,0.000000,0.000649,0.000649,0.000000,0.000649,0.000649,0.007792,0.000000,0.000000,0.000000,0.012338,...,0.003247,0.000000,0.020779,0.014935,0.005195,0.000649,0.005195,0.0,0.001

#### Q1. Run linear regression 'sale_price ~ gross_sq_feet+land_sq_feet+year_built' over the training data

## REsample1 is the training data

In [150]:
REsample1.head()

,borough,neighborhood,block,lot,easement,building_class_present,address,zip_code,commmercial_units,land_sq_feet,gross_sq_feet,year_built,sale_price,sale_date
217555,2,BATHGATE,3046,42,NaN,A1,2069 BATHGATE AVE,10457,0,1964,1424,1899,345376,2012-04-05
217599,2,BAYCHESTER,4708,48,NaN,A5,1110 EAST 214TH STREET,10469,0,1710,1813,1945,338000,2012-02-03
217605,2,BAYCHESTER,4715,61,NaN,A9,1118 221ST STREET,10469,0,2514,1690,1950,250000,2012-12-17
217609,2,BAYCHESTER,4724,21,NaN,A1,3453 KINGSLAND AVENUE,10469,0,2283,1588,1910,249600,2012-06-01
217610,2,BAYCHESTER,4725,45,NaN,A9,1453 HICKS STREET,10469,0,1710,1690,1945,313000,2012-09-20


In [151]:
REsample1.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5832 entries, 217555 to 301570
Data columns (total 14 columns):
borough                   5832 non-null int64
neighborhood              5832 non-null object
block                     5832 non-null int64
lot                       5832 non-null int64
easement                  0 non-null float64
building_class_present    5832 non-null object
address                   5832 non-null object
zip_code                  5832 non-null int64
commmercial_units         5832 non-null int64
land_sq_feet              5832 non-null int64
gross_sq_feet             5832 non-null int64
year_built                5832 non-null int64
sale_price                5832 non-null int64
sale_date                 5832 non-null object
dtypes: float64(1), int64(9), object(4)
memory usage: 683.4+ KB


In [152]:
lm = smf.ols(formula = 'sale_price~gross_sq_feet+land_sq_feet+year_built', data = 'REsample1').fit()
REsample1['Predicted-Sale_Price'] = lm.predict()
lm.summary()

TypeError: ignored

In [0]:
reg = smf.ols('sale_price~gross_sq_feet+land_sq_feet+year_built', data = 'REsample1').fit()

#### Q2. Add 311 data corresponding to the zip code of the house location to REsample1,2,3 
in order to characterize the house's neighborhood by merging the dataframes REsample1,2,3 with data311 by zip code. Name merged dataframes REsample1_311, REsample2_311, REsample3_311 
rename 311 columns to F1,F2,... for easier formulae engineering
but keep the original names as a dictionary {'F1':'APPLIANCE', ...}

In [0]:
REsample1_311 = REsample1.join(data311, on = 'zip_code')

In [154]:
REsample1_311.head()

,borough,neighborhood,block,lot,easement,building_class_present,address,zip_code,commmercial_units,land_sq_feet,gross_sq_feet,year_built,sale_price,sale_date,appliance,adopt-a-basket,air,animal,animal,animal,asbestos,best/site,beach/pool/sauna,bike,bike/roller/skate,blocked,boilers,bottled,bridge,broken,broken,building,building/use,bus,construction,calorie,city,collection,construction,consumer,...,school,senior,sewer,sidewalk,smoking,snow,special,special,special,sprinkler,squeegee,stalled,standing,standpipe,street,street,street,street,street,summer,sweeping/missed-inadequate,tattooing,taxi,traffic,traffic,trans,transportation,unleashed,unsanitary,unsanitary,unsanitary,urinating,vacant,vending,violation,water,water,water,window,x-ray
217555,2,BATHGATE,3046,42,NaN,A1,2069 BATHGATE AVE,10457,0,1964,1424,1899,345376,2012-04-05,0.007921,0.0,0.001980,0.002087,0.000000,0.000589,0.000321,0.000321,0.0,0.0,0.000054,0.014878,0.001659,0.0,0.000000,0.001124,0.0,0.0,0.007279,0.000054,0.000589,0.0,0.0,0.0,0.000054,0.007332,...,0.001713,0.0,0.006048,0.000749,0.000375,0.000268,0.001873,0.0,0.000375,0.0,0.0,0.000107,0.000161,0.0,0.013647,0.007332,0.000589,0.000268,0.000535,0.0,0.000107,0.000107,0.000428,0.000535,0.002622,0.0,0.0,0.000107,0.0,0.001124,0.000161,0.000268,0.000321,0.000428,0.000482,0.000803,0.000268,0.028311,0.000000,0.0
217599,2,BAYCHESTER,4708,48,NaN,A5,1110 EAST 214TH STREET,10469,0,1710,1813,1945,338000,2012-02-03,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.0,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.0,0.0,0.000101,0.010120,...,0.000101,0.0,0.026010,0.002024,0.000607,0.001316,0.005060,0.0,0.000911,0.0,0.0,0.000000,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.000000,0.000405,0.001012,0.011234,0.0,0.0,0.000810,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.046250,0.000101,0.0
217605,2,BAYCHESTER,4715,61,NaN,A9,1118 221ST STREET,10469,0,2514,1690,1950,250000,2012-12-17,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.0,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.0,0.0,0.000101,0.010120,...,0.000101,0.0,0.026010,0.002024,0.000607,0.001316,0.005060,0.0,0.000911,0.0,0.0,0.000000,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.000000,0.000405,0.001012,0.011234,0.0,0.0,0.000810,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.046250,0.000101,0.0
217609,2,BAYCHESTER,4724,21,NaN,A1,3453 KINGSLAND AVENUE,10469,0,2283,1588,1910,249600,2012-06-01,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.0,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.0,0.0,0.000101,0.010120,...,0.000101,0.0,0.026010,0.002024,0.000607,0.001316,0.005060,0.0,0.000911,0.0,0.0,0.000000,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.000000,0.000405,0.001012,0.011234,0.0,0.0,0.000810,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.046250,0.000101,0.0
217610,2,BAYCHESTER,4725,45,NaN,A9,1453 HICKS STREET,10469,0,1710,1690,1945,313000,2012-09-20,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.0,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.0,0.0,0.000101,0.010120,...,0.000101,0.0,0.026010,0.002024,0.000607,0.001316,0.005060,0.0,0.000911,0.0,0.0,0.000000,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.000000,0.000405,0.001012,0.011234,0.0,0.0,0.000810,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.046250,0.000101,0.0


In [155]:
REsample2_311 = REsample2.join(data311, on = 'zip_code')
REsample2_311.head()

,borough,neighborhood,block,lot,easement,building_class_present,address,zip_code,commmercial_units,land_sq_feet,gross_sq_feet,year_built,sale_price,sale_date,appliance,adopt-a-basket,air,animal,animal,animal,asbestos,best/site,beach/pool/sauna,bike,bike/roller/skate,blocked,boilers,bottled,bridge,broken,broken,building,building/use,bus,construction,calorie,city,collection,construction,consumer,...,school,senior,sewer,sidewalk,smoking,snow,special,special,special,sprinkler,squeegee,stalled,standing,standpipe,street,street,street,street,street,summer,sweeping/missed-inadequate,tattooing,taxi,traffic,traffic,trans,transportation,unleashed,unsanitary,unsanitary,unsanitary,urinating,vacant,vending,violation,water,water,water,window,x-ray
217600,2,BAYCHESTER,4710,1,NaN,A5,1127 EAST 215TH STREET,10469,0,2759,1998,1945,0,2012-12-20,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.0,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.0,0.0,0.000101,0.01012,...,0.000101,0.0,0.02601,0.002024,0.000607,0.001316,0.00506,0.0,0.000911,0.0,0.0,0.0,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.0,0.000405,0.001012,0.011234,0.0,0.0,0.00081,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.04625,0.000101,0.0
217601,2,BAYCHESTER,4710,1,NaN,A5,1127 EAST 215TH STREET,10469,0,2759,1998,1945,0,2012-03-27,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.0,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.0,0.0,0.000101,0.01012,...,0.000101,0.0,0.02601,0.002024,0.000607,0.001316,0.00506,0.0,0.000911,0.0,0.0,0.0,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.0,0.000405,0.001012,0.011234,0.0,0.0,0.00081,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.04625,0.000101,0.0
217603,2,BAYCHESTER,4715,22,NaN,A5,1156 EAST 221 STREET,10469,0,1842,1744,1950,180000,2012-12-19,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.0,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.0,0.0,0.000101,0.01012,...,0.000101,0.0,0.02601,0.002024,0.000607,0.001316,0.00506,0.0,0.000911,0.0,0.0,0.0,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.0,0.000405,0.001012,0.011234,0.0,0.0,0.00081,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.04625,0.000101,0.0
217612,2,BAYCHESTER,4730,10,NaN,A1,1320 EAST 222 STREET,10469,0,1938,1974,1901,232000,2012-08-23,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.0,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.0,0.0,0.000101,0.01012,...,0.000101,0.0,0.02601,0.002024,0.000607,0.001316,0.00506,0.0,0.000911,0.0,0.0,0.0,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.0,0.000405,0.001012,0.011234,0.0,0.0,0.00081,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.04625,0.000101,0.0
217614,2,BAYCHESTER,4730,45,NaN,A5,3418 GUNTHER AVENUE,10469,0,1710,1890,1950,335000,2012-12-19,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.0,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.0,0.0,0.000101,0.01012,...,0.000101,0.0,0.02601,0.002024,0.000607,0.001316,0.00506,0.0,0.000911,0.0,0.0,0.0,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.0,0.000405,0.001012,0.011234,0.0,0.0,0.00081,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.04625,0.000101,0.0


In [156]:
REsample3_311 = REsample3.join(data311, on = 'zip_code')
REsample3_311.head()

,borough,neighborhood,block,lot,easement,building_class_present,address,zip_code,commmercial_units,land_sq_feet,gross_sq_feet,year_built,sale_price,sale_date,appliance,adopt-a-basket,air,animal,animal,animal,asbestos,best/site,beach/pool/sauna,bike,bike/roller/skate,blocked,boilers,bottled,bridge,broken,broken,building,building/use,bus,construction,calorie,city,collection,construction,consumer,...,school,senior,sewer,sidewalk,smoking,snow,special,special,special,sprinkler,squeegee,stalled,standing,standpipe,street,street,street,street,street,summer,sweeping/missed-inadequate,tattooing,taxi,traffic,traffic,trans,transportation,unleashed,unsanitary,unsanitary,unsanitary,urinating,vacant,vending,violation,water,water,water,window,x-ray
204018,4,LAURELTON,12941,94,NaN,A2,131-43 FRANCIS LEWIS BLVD,11413,0,3560,1473,1940,0,2012-01-01,0.001934,0.0,0.002462,0.000000,0.000000,0.001055,0.000352,0.000352,0.0,0.0,0.000352,0.039388,0.000703,0.000176,0.000000,0.000000,0.0,0.0,0.038685,0.000000,0.000176,0.0,0.000176,0.0,0.018639,0.008089,...,0.000879,0.0,0.058555,0.001934,0.000000,0.000528,0.006858,0.000176,0.000528,0.0,0.0,0.0,0.002286,0.0,0.066292,0.101284,0.003165,0.002813,0.004924,0.0,0.000176,0.0,0.001055,0.002462,0.022859,0.0,0.0,0.000879,0.0,0.000528,0.000000,0.000000,0.004396,0.000000,0.000176,0.007209,0.001231,0.067698,0.000000,0.0
217598,2,BAYCHESTER,4707,70,NaN,A5,1134 EAST 213 STREET,10469,0,2320,1813,1950,335700,2012-05-17,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.000000,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.000000,0.0,0.000101,0.010120,...,0.000101,0.0,0.026010,0.002024,0.000607,0.001316,0.005060,0.000000,0.000911,0.0,0.0,0.0,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.0,0.000405,0.001012,0.011234,0.0,0.0,0.000810,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.046250,0.000101,0.0
217602,2,BAYCHESTER,4711,96,NaN,A5,1323 HICKS STREET,10469,0,1995,1701,1950,357500,2012-05-04,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.000000,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.000000,0.0,0.000101,0.010120,...,0.000101,0.0,0.026010,0.002024,0.000607,0.001316,0.005060,0.000000,0.000911,0.0,0.0,0.0,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.0,0.000405,0.001012,0.011234,0.0,0.0,0.000810,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.046250,0.000101,0.0
217604,2,BAYCHESTER,4715,56,NaN,A9,3840 LACONIA AVENUE,10469,0,2320,1690,1950,290000,2012-12-18,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.000000,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.000000,0.0,0.000101,0.010120,...,0.000101,0.0,0.026010,0.002024,0.000607,0.001316,0.005060,0.000000,0.000911,0.0,0.0,0.0,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.0,0.000405,0.001012,0.011234,0.0,0.0,0.000810,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.046250,0.000101,0.0
217606,2,BAYCHESTER,4717,26,NaN,A2,1460 OAKLEY STREET,10469,0,4750,2399,1930,375000,2012-11-29,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.000000,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.000000,0.0,0.000101,0.010120,...,0.000101,0.0,0.026010,0.002024,0.000607,0.001316,0.005060,0.000000,0.000911,0.0,0.0,0.0,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.0,0.000405,0.001012,0.011234,0.0,0.0,0.000810,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.046250,0.000101,0.0


#### Q3. Run the linear model with 311
We need to add 311 features in addition to the gross_sq_feet,land_sq_feet,year_built used before. So regression will include both - house characteristics as well as 311 features of the location
Train the model over the training sample REsample1_311 and test over the test sample REsample3_311,
report training and test R2

In [157]:
REsample1_311.head()

,borough,neighborhood,block,lot,easement,building_class_present,address,zip_code,commmercial_units,land_sq_feet,gross_sq_feet,year_built,sale_price,sale_date,appliance,adopt-a-basket,air,animal,animal,animal,asbestos,best/site,beach/pool/sauna,bike,bike/roller/skate,blocked,boilers,bottled,bridge,broken,broken,building,building/use,bus,construction,calorie,city,collection,construction,consumer,...,school,senior,sewer,sidewalk,smoking,snow,special,special,special,sprinkler,squeegee,stalled,standing,standpipe,street,street,street,street,street,summer,sweeping/missed-inadequate,tattooing,taxi,traffic,traffic,trans,transportation,unleashed,unsanitary,unsanitary,unsanitary,urinating,vacant,vending,violation,water,water,water,window,x-ray
217555,2,BATHGATE,3046,42,NaN,A1,2069 BATHGATE AVE,10457,0,1964,1424,1899,345376,2012-04-05,0.007921,0.0,0.001980,0.002087,0.000000,0.000589,0.000321,0.000321,0.0,0.0,0.000054,0.014878,0.001659,0.0,0.000000,0.001124,0.0,0.0,0.007279,0.000054,0.000589,0.0,0.0,0.0,0.000054,0.007332,...,0.001713,0.0,0.006048,0.000749,0.000375,0.000268,0.001873,0.0,0.000375,0.0,0.0,0.000107,0.000161,0.0,0.013647,0.007332,0.000589,0.000268,0.000535,0.0,0.000107,0.000107,0.000428,0.000535,0.002622,0.0,0.0,0.000107,0.0,0.001124,0.000161,0.000268,0.000321,0.000428,0.000482,0.000803,0.000268,0.028311,0.000000,0.0
217599,2,BAYCHESTER,4708,48,NaN,A5,1110 EAST 214TH STREET,10469,0,1710,1813,1945,338000,2012-02-03,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.0,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.0,0.0,0.000101,0.010120,...,0.000101,0.0,0.026010,0.002024,0.000607,0.001316,0.005060,0.0,0.000911,0.0,0.0,0.000000,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.000000,0.000405,0.001012,0.011234,0.0,0.0,0.000810,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.046250,0.000101,0.0
217605,2,BAYCHESTER,4715,61,NaN,A9,1118 221ST STREET,10469,0,2514,1690,1950,250000,2012-12-17,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.0,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.0,0.0,0.000101,0.010120,...,0.000101,0.0,0.026010,0.002024,0.000607,0.001316,0.005060,0.0,0.000911,0.0,0.0,0.000000,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.000000,0.000405,0.001012,0.011234,0.0,0.0,0.000810,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.046250,0.000101,0.0
217609,2,BAYCHESTER,4724,21,NaN,A1,3453 KINGSLAND AVENUE,10469,0,2283,1588,1910,249600,2012-06-01,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.0,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.0,0.0,0.000101,0.010120,...,0.000101,0.0,0.026010,0.002024,0.000607,0.001316,0.005060,0.0,0.000911,0.0,0.0,0.000000,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.000000,0.000405,0.001012,0.011234,0.0,0.0,0.000810,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.046250,0.000101,0.0
217610,2,BAYCHESTER,4725,45,NaN,A9,1453 HICKS STREET,10469,0,1710,1690,1945,313000,2012-09-20,0.004757,0.0,0.001822,0.003643,0.000101,0.000405,0.001316,0.000607,0.0,0.0,0.000405,0.070337,0.001012,0.0,0.000101,0.002935,0.0,0.0,0.021455,0.000304,0.000708,0.0,0.0,0.0,0.000101,0.010120,...,0.000101,0.0,0.026010,0.002024,0.000607,0.001316,0.005060,0.0,0.000911,0.0,0.0,0.000000,0.002125,0.0,0.035624,0.086428,0.003441,0.001316,0.004655,0.0,0.001417,0.000000,0.000405,0.001012,0.011234,0.0,0.0,0.000810,0.0,0.003036,0.000405,0.000202,0.001619,0.000202,0.000101,0.004959,0.000708,0.046250,0.000101,0.0


In [159]:
cols = data311.columns
form = 'sale_price ~ gross_sq_feet+land_sq_feet+year_built'+'+'+'+'.join(REsample1_311.columns[14:])
print(form)


sale_price ~ gross_sq_feet+land_sq_feet+year_built+appliance+adopt-a-basket+air+animal+animal+animal+asbestos+best/site+beach/pool/sauna+bike+bike/roller/skate+blocked+boilers+bottled+bridge+broken+broken+building+building/use+bus+construction+calorie+city+collection+construction+consumer+cranes+curb+dof+dpr+damaged+dead+derelict+derelict+derelict+dirty+disorderly+drinking+drinking+eap+electric+electrical+elevator+emergency+fire+fire+fire+fire+fire+fire+food+food+for+found+general+general+general+graffiti+heating+harboring+hazardous+hazmat+highway+highway+highway+highway+homeless+homeless+illegal+illegal+illegal+illegal+illegal+indoor+indoor+industrial+investigations+lead+legal+lifeguard+literature+litter+maintenance+miscellaneous+missed+mold+municipal+nonconst+noise+noise+noise+noise+noise+noise+noise+non-residential+open+other+overflowing+overgrown+paint+paint/plaster+plumbing+panhandling+plant+plumbing+poison+portable+posting+public+public+public+public+radioactive+rangehood+recycli

In [161]:
lm  = smf.ols(formula = 'sale_price ~ gross_sq_feet+land_sq_feet+year_built'+'+'+'+'.join(REsample1_311.columns[14:]), data = REsample1_311).fit()
REsample1_311['PREDICTIONS'] = lm.predict()
lm.summary()

SyntaxError: ignored

In [0]:
#R2 over training
print("R2_Score for Training Data: {}".format(r2_score(REsample1_311.sale_price, REsample1_311.PREDICTIONS)))


In [0]:
REsample3_311['test_PREDICTIONS'] =lm.predict(REsample3_311)
print(lm.summary())

In [0]:
print("R2_Score for Testing Data: {}".format(r2_score(REsample3_311.sale_price, REsample3_311.test_PREDICTIONS)))

#### Q4. Apply regularization
Train Ridge and Lasso over REsample1_311 with an arbitrary default parameter $\alpha=1$.
Then evaluate their out-of-sample performance over REsample3_311
Make sure to scale the features in all three samples first, learning the scaler over the training sample

#### Q5. Perform parameter selection for Lasso and Ridge over validation sample
try exponential range -exp(10)...exp(20) with 100 values in it

Visualize the validation R2 vs alpha

For the optimal alpha evaluate performance of the models over the test set

#### Task 2. Model Mean income of the zip code population using 311 profile of the neighborhood

In [0]:
#load the data
dataIncome=pd.read_csv(folder+"data/MeanZip.csv",index_col=0)
dataIncome=dataIncome.apply(pd.to_numeric, errors='coerce')
dataIncome=dataIncome.loc[dataIncome.Mean>0]
dataIncome.head()

In [0]:
#split into training (1), validation (2) and test (3) samples
np.random.seed(2018)
splitind=np.random.uniform(size=len(dataIncome))
dataIncome1=dataIncome.loc[splitind<=0.4]
dataIncome2=dataIncome.loc[(splitind>0.4)&(splitind<0.7)]
dataIncome3=dataIncome.loc[(splitind>=0.7)]

#### Q1. Perform regularied regression of Mean income of the zip code vs its 311 profile 
Train Ridge and Lasso over dataIncome1 with an arbitrary default parameter $\alpha=100$.
Then evaluate their out-of-sample performance over dataIncome3
Make sure to scale the features in all three samples first, learning the scaler over the training sample

#### Q2. Perform parameter selection for Lasso and Ridge over validation sample
try exponential range exp(-10)...exp(20) with 100 values in it

Visualize the validation R2 vs alpha

For the optimal alpha evaluate performance of the models over the test set. 

Why do you think performance does not look that good, especially for one of the models?